In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, JavascriptException
import time
import re
import sys
import pandas as pd
total_review= []
visited_urls = set(pd.read_csv('visited_url.csv')['url'])
url_pattern = r"https?:\/\/.*?\.html|\/\/.*?\.html"

options = Options()
options.add_argument("--no-sandbox") 
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

url = "https://www.daraz.com.np/products/xbox-360-console-pc-windows-usb-wired-joystick-gamepad-joypad-controller-i111995726-s1030293946.html"
visited_urls.add(url)
# Load page twice (key part that works)
driver.get(url)
wait = WebDriverWait(driver, 7)
max_retries = 5
attempt = 0
while attempt < max_retries:
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-spm-anchor-id*="a2a0e.pdp_revamp"]')))
        break
    except TimeoutException:
        attempt += 1
        driver.get(url)
        time.sleep(3)
if attempt==max_retries:
    print("Element not found after several retries.")
    sys.exit() 


driver.get(url)  # Second load triggers content
driver.implicitly_wait(5)
driver.maximize_window()
# User interaction
actions = ActionChains(driver)
actions.move_by_offset(100, 100).perform()
driver.execute_script("""window.scrollTo({top: 1300,behavior: 'smooth'});""")

time.sleep(1)
total_review.extend(get_review(driver))
driver.execute_script("window.scrollTo({top: document.body.scrollHeight - 300, behavior: 'smooth'});")
scrape_iteration = 7
iteration_count= 0
for outter_i in range(scrape_iteration):
    try:
        print(f"Product no:{iteration_count}")
        iteration_count +=1
        if (iteration_count % 5) == 0:
            prev_review = pd.read_csv('scraped_review.csv')
            new_reviews_df = pd.DataFrame(total_review, columns=['review'])
            combined = pd.concat([prev_review, new_reviews_df], ignore_index=True)
            combined.to_csv('scraped_review.csv', index=False)
            total_review = []
        scroll_count = 0
        while True:
            try:
                if scroll_count > 10:
                    raise JavascriptException
                driver.execute_script(f"window.scrollTo({{top: {1000 + scroll_count * 800}, behavior: 'smooth'}});")
                time.sleep(2)  # Let the page load

                # Try to find the target element
                next_item_mark = driver.find_element(By.CSS_SELECTOR, 'a[data-spm-anchor-id="a2a0e.pdp_revamp.recommendation_2.1"]')
                break  # Found it, exit loop

            except NoSuchElementException:
                scroll_count += 1

            except JavascriptException:
                print("Reached end of page or script failed.")
                sys.exit()
        driver.execute_script("""arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });""", next_item_mark)
        driver.execute_script(""" window.scrollBy({top: 800,behavior: 'smooth'});""")
        found_match = False
        for j in range(4):
            next_item= driver.find_element(By.CSS_SELECTOR, f'a[data-spm-anchor-id="a2a0e.pdp_revamp.recommendation_2.{j+1}"]')
            match = re.search(url_pattern, next_item.get_attribute("href"))
            if match:
                clean_href = "https:" + match.group() if match.group().startswith("//") else match.group()
                if clean_href not in visited_urls:
                    print("found a match: recommended product")
                    visited_urls.add(clean_href)
                    next_item.click()
                    max_retries = 5
                    attempt = 0
                    while attempt < max_retries:
                        try:
                            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-spm-anchor-id*="a2a0e.pdp_revamp"]')))
                            break
                        except TimeoutException:
                            attempt += 1
                            driver.get(clean_href)
                            time.sleep(3)
                    if attempt == max_retries:
                        print("Element not found after several retries.")
                        sys.exit()
                    driver.execute_script("""window.scrollTo({top: 3300,behavior: 'smooth'});""")
                    total_review.extend(get_review(driver))
                    found_match = True
                    break
            else:
                print("No match found.")
        if not found_match:
            print("No matches found in all 4 iterations, going back...")
            driver.back()
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-spm-anchor-id*="a2a0e.pdp_revamp"]')))
            except TimeoutException:
                print('Time out Exception')
                sys.exit()
    except Exception as e:
        print(f'Some error occurred:{e}')
        print('Ending the scraping process!')
        print('Saving till the last checkpoint!')
        break
prev_review = pd.read_csv('scraped_review.csv')
new_reviews_df = pd.DataFrame(total_review, columns=['review'])
combined = pd.concat([prev_review, new_reviews_df], ignore_index=True)
combined.to_csv('scraped_review.csv', index=False)
visited_urls_df = pd.DataFrame(list(visited_urls), columns=['url'])
visited_urls_df.to_csv('visited_url.csv', index=False)
driver.quit()

No next button found. Stopping.
Product no:0
found a match: recommended product
Click intercepted, attempt 1
No next button found. Stopping.
Product no:1
found a match: recommended product
No next button found. Stopping.
Product no:2
found a match: recommended product
Click intercepted, attempt 1
No next button found. Stopping.
Product no:3
found a match: recommended product
No next button found. Stopping.
Product no:4
found a match: recommended product
Click intercepted, attempt 1
No next button found. Stopping.
Product no:5
found a match: recommended product
No next button found. Stopping.
Product no:6
found a match: recommended product
No next button found. Stopping.
Product no:7
found a match: recommended product
Some error occurred:HTTPConnectionPool(host='localhost', port=53968): Read timed out. (read timeout=120)
Ending the scraping process!
Saving till the last checkpoint!


In [2]:
def get_review(driver):
    review_list=[]
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.content")))
        review_section = driver.find_element(By.CSS_SELECTOR,"div.pdp-mod-review[data-spm='ratings_reviews']")
        review = driver.find_elements(By.CSS_SELECTOR, "div.content")
        for index, review_el in enumerate(review):
            is_inside = driver.execute_script("""return arguments[1].contains(arguments[0]);""", review_el ,review_section)
            if is_inside and "thank you" not in review_el.text.lower() and re.search(r'[a-z]', review_el.text.lower()) and not re.search(r'\d\.\d/5\n\d+ Ratings', review_el.text.lower()):
                review_list.append(review_el.text)
            
    except Exception as e:
        print(f"Error finding value{e}")
    
    #finding items review    
    while True:
        try:
            time.sleep(4)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.content")))
            review_section = driver.find_element(By.CSS_SELECTOR,"div.pdp-mod-review[data-spm='ratings_reviews']")
            next_button = driver.find_element(By.CSS_SELECTOR, ".next-btn.next-btn-normal.next-btn-medium.next-pagination-item.next")
            is_inside = driver.execute_script("""return arguments[1].contains(arguments[0]);""", next_button, review_section)
            #Checking if the next button is either disabled or not of the review section
            if next_button.get_attribute("disabled") or not is_inside:
                raise NoSuchElementException("Next button is disabled.")
            
            
                    
            driver.execute_script("""arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });""", next_button)
            driver.execute_script(""" window.scrollBy({top: +800,behavior: 'smooth'});""")
            
            #clicks if safe 
            if not safe_click(driver, next_button):
                print("Failed to click after all attempts")
                continue
            time.sleep(2)
            review = driver.find_elements(By.CSS_SELECTOR, "div.content")
            for index, review_el in enumerate(review):
                is_inside = driver.execute_script("""return arguments[1].contains(arguments[0]);""", review_el ,review_section)
                if is_inside and "thank you" not in review_el.text.lower() and re.search(r'[a-z]', review_el.text.lower()) and not re.search(r'\d\.\d/5\n\d+ Ratings', review_el.text.lower()):
                    review_list.append(review_el.text)
                
        except NoSuchElementException:
            print("No next button found. Stopping.")
            break
    return review_list

In [3]:
def safe_click(driver, element, max_attempts=3):
    """Safely click an element with multiple fallback strategies"""
    for attempt in range(max_attempts):
        try:
            element.click()
            return True
        except ElementClickInterceptedException:
            print(f"Click intercepted, attempt {attempt + 1}")
            for inner_i in range(3):
                driver.execute_script(f"window.scrollTo({{top: {1000 + inner_i*800}, behavior: 'smooth'}});")
                time.sleep(2)  # Wait for lazy loading
            if attempt == 0:
                # First attempt: scroll element into better view
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
                time.sleep(1)
            elif attempt == 1:
                # Second attempt: scroll a bit more
                driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                time.sleep(1)
            else:
                # Final attempt: JavaScript click as last resort
                driver.execute_script("arguments[0].click();", element)
                return True
                
    return False

In [5]:
import pandas as pd
df = pd.read_csv('scraped_review.csv')
df

,review
0,verrrrry good product you must give it a try.
1,price anusar tikai xa good tks daraz\n
2,awesome🖤🖤
3,Ramro cha. Sabai accessories haru ni ramro san...
4,Okay xa
...,...
2364,I paid extra for the white controller but they...
2365,We sincerely apologize for the mix-up with you...
2366,4.5/5\n22 Ratings\n18\n1\n0\n1\n2
2367,5.0/5\n3 Ratings\n3\n0\n0\n0\n0


In [6]:
set_df = set(df['review'])

In [11]:
import re
another_set =set()
for item in set_df:
    if re.search(r'[a-z]',str(item).lower()):
        another_set.add(item) 

In [12]:
another_set

{"Thanks so much for taking the time to leave us rating – it's much appreciated",
 ' light weight budget friendly 💵',
 'met my expectations tq so much  ',
 'Impressive looks. Great deal at this price. ',
 " nice your  product I'm so happy ",
 'recently bought this mouse, feels great in hand, smooth xa precise track garxa, packaging was well-done. Definitely a great purchase ',
 'so good with price with quality and delivery  ',
 'What do you think of the quality and appearance?its good but it is very hard to control and i my opinion the buttons are a little bit too closer also it stops working for 15s to 25s in between of a game so overall its ok ',
 'Good for Money.Highly recomend it. ',
 'dammii',
 'paisa anusar thik chha ',
 'Nice shoes 👟😘 ',
 'good product and reasonable price but one thing is unexpected 😔, there is no visibility of keyboard number and letter when  rgb is ON OR high light . if you can change  ,you will do  in bold font or bulk letter.',
 'Too satisfied I already bou

In [13]:
len(another_set)

2219

In [14]:
4.6/5\n186 Ratings\n136\n36\n9\n2\n3
5.0/5\n3 Ratings\n3\n0\n0\n0\n0
r'[0-5].[0-5]/5\n[0-9]+ Ratings'

SyntaxError: unexpected character after line continuation character (3958214960.py, line 1)